In [1]:
DEBUG = False

In [2]:
import numpy as np
import random
import pandas as pd
import os

In [3]:
from collections import defaultdict
# import datatable as dt
import lightgbm as lgb
# from matplotlib import pyplot as plt
import riiideducation
# from sklearn.metrics import roc_auc_score
import gc

_ = np.seterr(divide='ignore', invalid='ignore')

In [4]:
clfs = []

import joblib
clfs.append(joblib.load('../input/riiid-inference-part/lgb3.pkl'))


In [5]:
target = 'answered_correctly'
pickles_path = '../input/riiid-pickle'

In [6]:
lagtime_in_lecture_mean = 20061832.085456338
lag_elapsed_time_mean = 20416566.858529676
lagtime_mean = 20441941.566456214
lagtime_mean2 = 40370652.30368494
lagtime_mean3 = 60008399.99610778
prior_question_elapsed_time_mean = 13005.0810546875

In [7]:
# origin
features_dict = {
    #'user_id',
    'timestamp':'float16',#
    'user_interaction_count':'int16',
    'user_interaction_timestamp_mean':'float32',
    'lagtime':'float32',#
    'lagtime2':'float32',
    'lagtime3':'float32',
    #'lagtime_mean':'int32',
    'content_id':'int16',
    'task_container_id':'int16',
    'user_lecture_sum':'int16',#
    'user_lecture_lv':'float16',##
    'prior_question_elapsed_time':'float32',#
    'delta_prior_question_elapsed_time':'int32',#
    'user_correctness':'float16',#
    'user_uncorrect_count':'int16',#
    'user_correct_count':'int16',#
    #'content_correctness':'float16',
    'content_correctness_std':'float16',
    'content_correct_count':'int32',
    'content_uncorrect_count':'int32',#
    'content_elapsed_time_mean':'float16',
    'content_had_explanation_mean':'float16',
    'content_explation_false_mean':'float16',
    'content_explation_true_mean':'float16',
    'task_container_correctness':'float16',
    'task_container_std':'float16',
    'task_container_cor_count':'int32',#
    'task_container_uncor_count':'int32',#
    'attempt_no':'int8',#
    'part':'int8',
    'part_correctness_mean':'float16',
    'part_correctness_std':'float16',
    'part_uncor_count':'int32',
    'part_cor_count':'int32',
    'tags0': 'int8',
    'tags1': 'int8',
    'tags2': 'int8',
    'tags3': 'int8',
    'tags4': 'int8',
    'tags5': 'int8',
   # 'tags6': 'int8',
   # 'tags7': 'int8',
#     'tags0_correctness_mean':'float16',
#     'tags1_correctness_mean':'float16',
#     'tags2_correctness_mean':'float16',
#     'tags4_correctness_mean':'float16',
#     'bundle_id':'int16',
#     'bundle_correctness_mean':'float16',
#     'bundle_uncor_count':'int32',
#     'bundle_cor_count':'int32',
    'part_bundle_id':'int32',
    'content_sub_bundle':'int8',
    'prior_question_had_explanation':'int8',
    'explanation_mean':'float16', #
    #'explanation_var',#
    'explanation_false_count':'int16',#
    'explanation_true_count':'int16',#
   # 'community':'int8',
#     'part_1',
#     'part_2',
#     'part_3',
#     'part_4',
#     'part_5',
#     'part_6',
#     'part_7',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
}
categorical_columns= [
    #'user_id',
    'content_id',
    'task_container_id',
    'part',
   # 'community',
    'tags0',
    'tags1',
    'tags2',
    'tags3',
    'tags4',
    'tags5',
    #'tags6',
    #'tags7',
    #'bundle_id',
    'part_bundle_id',
    'content_sub_bundle',
    'prior_question_had_explanation', 
#     'part_1',
#     'part_2',
#     'part_3',
#     'part_4',
#     'part_5',
#     'part_6',
#     'part_7',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
]

features=list(features_dict.keys())


In [8]:
###### Value>100000
features_dict = {
#  'lagtime': 'float32', # curr
 'lagtime2': 'float32', #
 'lagtime3': 'float32',
 'content_id': 'int16',
#  'task_container_id': 'int16', # curr
 'prior_question_elapsed_time': 'float32',
 'user_correctness': 'float16',
#  'user_uncorrect_count': 'int16', # curr
 'user_correct_count': 'int16',
 'content_correctness_std': 'float16',
 'content_correct_count': 'int32',
 'content_uncorrect_count': 'int32',
 'content_had_explanation_mean': 'float16', #
#  'content_explation_false_mean': 'float16', # curr
 'content_explation_true_mean': 'float16',
 'attempt_no': 'int8',
#  'part': 'int8', # curr
 'part_correctness_mean': 'float16',
 'part_bundle_id': 'int32',
 'explanation_false_count': 'int16',
 'lagtime_in_lecture': 'float32',
 'content_lagtime': 'float32',
#  'lag_elapsed_time': 'float32', # curr
 'attempt_no_mean': 'float16',
 'lagtime1_2': 'float32',
 'lagtime2_3': 'float32',
 'user_part_correctness': 'float16',
#  'user_part_rate': 'float16',
 'part_lagtime': 'float16',
 'user_lecture_sum': 'float16',
 'user_lecture_lv': 'int16',
#  'community': 'int16',
#  'community_correctness_mean': 'float16',
#  'lecture_of_day': 'int16',
#  'user_interaction_lagtime_mean': 'float32',
#  'lagtime_cumsum': 'int16'
}

features=list(features_dict.keys())

delete_col = set(categorical_columns) - set(features)
categorical_columns = list(set(categorical_columns) - delete_col)
print(categorical_columns)

['content_id', 'part_bundle_id']


# Inference

In [9]:
print('start Inference')

start Inference


In [10]:
# lagtime_sum_dict = lagtime_agg['sum'].astype('int16').to_dict(defaultdict(int))
# lagtime_count_dict = lagtime_agg['count'].astype('int16').to_dict(defaultdict(int))

# del lagtime_agg

In [11]:
questions_df = pd.read_pickle(os.path.join(pickles_path, 'questions_df.pkl'))
# lecture情報取り込む際に邪魔になる
questions_df.drop('part', axis=1)

part_lecture_user_agg = pd.read_pickle('../input/riiid-inference-part/part_lecture_user_agg.pkl')
part_lecture_user_sum_dict = part_lecture_user_agg['sum'].astype('int16').to_dict(defaultdict(int))
part_lecture_user_count_dict = part_lecture_user_agg['count'].astype('int16').to_dict(defaultdict(int))

del part_lecture_user_agg

In [12]:
max_timestamp_u_part = pd.read_pickle('../input/riiid-inference-part/max_timestamp_u_part_org.pkl')
max_timestamp_u_part_dict = max_timestamp_u_part['timestamp'].to_dict(defaultdict(int))
del max_timestamp_u_part

part_user_agg = pd.read_pickle('../input/riiid-inference-part/part_user_agg.pkl')
part_user_sum_dict = part_user_agg['sum'].astype('int16').to_dict(defaultdict(int))
part_user_count_dict = part_user_agg['count'].astype('int16').to_dict(defaultdict(int))

del part_user_agg

In [13]:
max_timestamp_u_content = pd.read_pickle(os.path.join(pickles_path, 'max_timestamp_u_content.pkl'))
max_timestamp_u_content_dict = max_timestamp_u_content['timestamp'].to_dict(defaultdict(int))
del max_timestamp_u_content

In [14]:
user_agg = pd.read_pickle(os.path.join(pickles_path, 'user_agg.pkl'))
user_sum_dict = user_agg['sum'].astype('int16').to_dict(defaultdict(int))
user_count_dict = user_agg['count'].astype('int16').to_dict(defaultdict(int))
# user_sum_dict2 = user_agg2['sum'].astype('int16').to_dict(defaultdict(int))
# user_count_dict2 = user_agg2['count'].astype('int16').to_dict(defaultdict(int))

del user_agg

In [15]:

#user_var_dict = user_agg['var'].astype('float16').to_dict(defaultdict(int))

# content_sum_dict = content_agg['sum'].astype('int32').to_dict(defaultdict(int))
# content_count_dict = content_agg['count'].astype('int32').to_dict(defaultdict(int))

#del user_agg2
#del content_agg
gc.collect()

task_container_agg = pd.read_pickle(os.path.join(pickles_path, 'task_container_agg.pkl'))
task_container_sum_dict = task_container_agg['sum'].astype('int32').to_dict(defaultdict(int))
task_container_count_dict = task_container_agg['count'].astype('int32').to_dict(defaultdict(int))
task_container_std_dict = task_container_agg['var'].astype('float16').to_dict(defaultdict(int))

explanation_agg = pd.read_pickle(os.path.join(pickles_path, 'explanation_agg.pkl'))
explanation_sum_dict = explanation_agg['sum'].astype('int16').to_dict(defaultdict(int))
explanation_count_dict = explanation_agg['count'].astype('int16').to_dict(defaultdict(int))
#explanation_var_dict = explanation_agg['var'].astype('float16').to_dict(defaultdict(int))
del task_container_agg
del explanation_agg
gc.collect()

0

In [16]:
#task_container_std_dict

In [17]:
# user_lecture_sum_dict = user_lecture_agg['sum'].astype('int16').to_dict(defaultdict(int))
# user_lecture_count_dict = user_lecture_agg['count'].astype('int16').to_dict(defaultdict(int))

#lagtime_mean_dict = lagtime_agg['mean'].astype('int32').to_dict(defaultdict(int))
#del prior_question_elapsed_time_agg
# del user_lecture_agg
#del lagtime_agg
gc.collect()

40

In [18]:
max_timestamp_u = pd.read_pickle(os.path.join(pickles_path, 'max_timestamp_u.pkl'))
max_timestamp_u2 = pd.read_pickle(os.path.join(pickles_path, 'max_timestamp_u2.pkl'))
max_timestamp_u3 = pd.read_pickle(os.path.join(pickles_path, 'max_timestamp_u3.pkl'))
user_prior_question_elapsed_time = pd.read_pickle(os.path.join(pickles_path, 'user_prior_question_elapsed_time.pkl'))

In [19]:
max_timestamp_u_dict=max_timestamp_u.set_index('user_id').to_dict()
max_timestamp_u_dict2=max_timestamp_u2.set_index('user_id').to_dict()
max_timestamp_u_dict3=max_timestamp_u3.set_index('user_id').to_dict()
user_prior_question_elapsed_time_dict=user_prior_question_elapsed_time.set_index('user_id').to_dict()
#del question_elapsed_time_agg
del max_timestamp_u
del max_timestamp_u2
del max_timestamp_u3
del user_prior_question_elapsed_time
gc.collect()

0

In [20]:
max_timestamp_u_dict_c = max_timestamp_u_dict.copy() # lagtime_in_lecture

In [21]:
attempt_no_agg = pd.read_pickle(os.path.join(pickles_path, 'attempt_no_agg.pkl'))
attempt_no_sum_dict = attempt_no_agg['sum'].to_dict(defaultdict(int))

attempt_cum_agg = pd.read_pickle(os.path.join(pickles_path, 'attempt_cum_agg.pkl'))
attempt_cum_sum_dict = attempt_cum_agg['sum'].astype('int16').to_dict(defaultdict(int))
attempt_cum_count_dict = attempt_cum_agg['count'].astype('int16').to_dict(defaultdict(int))

del attempt_no_agg
gc.collect()

0

In [22]:
def get_max_attempt(user_id,content_id):
    k = (user_id,content_id)

    if k in attempt_no_sum_dict.keys():
        attempt_no_sum_dict[k]+=1
        return attempt_no_sum_dict[k]

    attempt_no_sum_dict[k] = 1
    return attempt_no_sum_dict[k]

In [23]:
def get_content_lagtime(user_id, content_id, timestamp):
    k = (user_id,content_id)

    if k in max_timestamp_u_content_dict.keys():
        tmp = max_timestamp_u_content_dict[k]
        max_timestamp_u_content_dict[k]+=timestamp
        return timestamp-tmp

    max_timestamp_u_content_dict[k] = timestamp
    return -1

# SAKT Part II

In [24]:
#HDKIM SAKT
import joblib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# MAX_SEQ = 100

In [25]:
MAX_SEQ = 250 # 250
ACCEPTED_USER_CONTENT_SIZE = 4
EMBED_SIZE = 128
BATCH_SIZE = 64
DROPOUT = 0.1

In [26]:
skills = joblib.load("/kaggle/input/riiid-sakt-model-dataset-public/skills.pkl.zip")
n_skill = len(skills)
group = joblib.load("/kaggle/input/riiid-sakt-model-dataset-public/group.pkl.zip")

In [27]:
class FFN(nn.Module):
    def __init__(self, state_size = 200, forward_expansion = 1, bn_size=MAX_SEQ - 1, dropout=0.2):
        super(FFN, self).__init__()
        self.state_size = state_size
        
        self.lr1 = nn.Linear(state_size, forward_expansion * state_size)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(bn_size)
        self.lr2 = nn.Linear(forward_expansion * state_size, state_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.relu(self.lr1(x))
        x = self.bn(x)
        x = self.lr2(x)
        return self.dropout(x)


def future_mask(seq_length):
    future_mask = (np.triu(np.ones([seq_length, seq_length]), k = 1)).astype('bool')
    return torch.from_numpy(future_mask)

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, heads = 8, dropout = DROPOUT, forward_expansion = 1):
        super(TransformerBlock, self).__init__()
        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=heads, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.layer_normal = nn.LayerNorm(embed_dim)
        self.ffn = FFN(embed_dim, forward_expansion = forward_expansion, dropout=dropout)
        self.layer_normal_2 = nn.LayerNorm(embed_dim)
        

    def forward(self, value, key, query, att_mask):
        att_output, att_weight = self.multi_att(value, key, query, attn_mask=att_mask)
        att_output = self.dropout(self.layer_normal(att_output + value))
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]
        x = self.ffn(att_output)
        x = self.dropout(self.layer_normal_2(x + att_output))
        return x.squeeze(-1), att_weight
    
class Encoder(nn.Module):
    def __init__(self, n_skill, max_seq=100, embed_dim=128, dropout = DROPOUT, forward_expansion = 1, num_layers=1, heads = 8):
        super(Encoder, self).__init__()
        self.n_skill, self.embed_dim = n_skill, embed_dim
        self.embedding = nn.Embedding(2 * n_skill + 1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq - 1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)
        self.layers = nn.ModuleList([TransformerBlock(embed_dim, forward_expansion = forward_expansion) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, question_ids):
        device = x.device
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
        pos_x = self.pos_embedding(pos_id)
        x = self.dropout(x + pos_x)
        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = self.e_embedding(question_ids)
        e = e.permute(1, 0, 2)
        for layer in self.layers:
            att_mask = future_mask(e.size(0)).to(device)
            x, att_weight = layer(e, x, x, att_mask=att_mask)
            x = x.permute(1, 0, 2)
        x = x.permute(1, 0, 2)
        return x, att_weight


class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=100, embed_dim=128, dropout = DROPOUT, forward_expansion = 1, enc_layers=1, heads = 8):
        super(SAKTModel, self).__init__()
        self.encoder = Encoder(n_skill, max_seq, embed_dim, dropout, forward_expansion, num_layers=enc_layers)
        self.pred = nn.Linear(embed_dim, 1)
        
    def forward(self, x, question_ids):
        x, att_weight = self.encoder(x, question_ids)
        x = self.pred(x)
        return x.squeeze(-1), att_weight

In [28]:
def create_model():
    return SAKTModel(n_skill, max_seq=MAX_SEQ, embed_dim=EMBED_SIZE, forward_expansion=1, enc_layers=1, heads=8, dropout=0.1)

is_gpu = torch.cuda.is_available()
if is_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

if MAX_SEQ!=180:
    MODEL_PATH = '../input/sakt-max-seq/sakt.pth'
else:
    MODEL_PATH = '/kaggle/input/sakt-model/sakt.pth'

sakt_model = create_model()
if is_gpu:
    sakt_model.load_state_dict(torch.load(MODEL_PATH))
else:
    sakt_model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
sakt_model.to(device)

SAKTModel(
  (encoder): Encoder(
    (embedding): Embedding(27047, 128)
    (pos_embedding): Embedding(249, 128)
    (e_embedding): Embedding(13524, 128)
    (layers): ModuleList(
      (0): TransformerBlock(
        (multi_att): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (dropout): Dropout(p=0.1, inplace=False)
        (layer_normal): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (ffn): FFN(
          (lr1): Linear(in_features=128, out_features=128, bias=True)
          (relu): ReLU()
          (bn): BatchNorm1d(249, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (lr2): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (layer_normal_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pred): Linear(in_features=128, out_featu

In [29]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, n_skill, max_seq=100):
        super(TestDataset, self).__init__()
        self.samples, self.user_ids, self.test_df = samples, [x for x in test_df["user_id"].unique()], test_df
        self.n_skill, self.max_seq = n_skill, max_seq

    def __len__(self):
        return self.test_df.shape[0]
    
    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]
        
        user_id = test_info['user_id']
        target_id = test_info['content_id']
        
        content_id_seq = np.zeros(self.max_seq, dtype=int)
        answered_correctly_seq = np.zeros(self.max_seq, dtype=int)
        
        if user_id in self.samples.index:
            content_id, answered_correctly = self.samples[user_id]
            
            seq_len = len(content_id)
            
            if seq_len >= self.max_seq:
                content_id_seq = content_id[-self.max_seq:]
                answered_correctly_seq = answered_correctly[-self.max_seq:]
            else:
                content_id_seq[-seq_len:] = content_id
                answered_correctly_seq[-seq_len:] = answered_correctly
                
        x = content_id_seq[1:].copy()
        x += (answered_correctly_seq[1:] == 1) * self.n_skill
        
        questions = np.append(content_id_seq[2:], [target_id])
        
        return x, questions

In [30]:
# model = lgb.Booster(model_file='../input/riiid-lgbm-starter/model.txt')
env = riiideducation.make_env()

In [31]:
iter_test = env.iter_test()
prior_test_df = None

In [32]:
 N=[0.4,0.6]

In [33]:
qdf = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
qdf = qdf[['question_id', 'part']]
qdf.columns = ['content_id', 'part']
qdf['content_type_id'] = 0

ldf = pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')
ldf = ldf[['lecture_id', 'part']]
ldf.columns = ['content_id', 'part']
ldf['content_type_id'] = 1

# key_diff = set(lectures_df.content_id).difference(questions_df.content_id)
# where_diff = lectures_df.content_id.isin(key_diff)

content_df = pd.concat([qdf, ldf])
content_df['join_key'] = content_df['content_type_id'].astype('str') + '_' + content_df['content_id'].astype('str')
content_df = content_df[['join_key', 'part']]

In [34]:
%%time

for (test_df, sample_prediction_df) in iter_test:    
    if prior_test_df is not None:
        prior_test_df[target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[target] != -1].reset_index(drop=True)
        #prior_test_df = prior_test_df[prior_test_df[target] != -1]
        prior_test_df['prior_question_had_explanation'].fillna(False, inplace=True)       
        prior_test_df.prior_question_had_explanation=prior_test_df.prior_question_had_explanation.astype('int8')
        
        #HDKIM SAKT State Update
        prev_group = prior_test_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))
        for prev_user_id in prev_group.index:
            prev_group_content = prev_group[prev_user_id][0]
            prev_group_ac = prev_group[prev_user_id][1]
            if prev_user_id in group.index:
                group[prev_user_id] = (np.append(group[prev_user_id][0],prev_group_content), 
                                       np.append(group[prev_user_id][1],prev_group_ac))
            else:
                group[prev_user_id] = (prev_group_content,prev_group_ac)
            if len(group[prev_user_id][0])>MAX_SEQ:
                new_group_content = group[prev_user_id][0][-MAX_SEQ:]
                new_group_ac = group[prev_user_id][1][-MAX_SEQ:]
                group[prev_user_id] = (new_group_content,new_group_ac)

        #HDKIMHDKIM
    
        user_ids = prior_test_df['user_id'].values
        timestamps = prior_test_df['timestamp'].values
        #content_ids = prior_test_df['content_id'].values
        #task_container_ids = prior_test_df['task_container_id'].values
        #prior_question_had_explanations = prior_test_df['prior_question_had_explanation'].values
        targets = prior_test_df[target].values
        attempt_nos = prior_test_df["attempt_no"].values
        parts = prior_test_df["part"].values
        
        
        for user_id, answered_correctly, timestamp, attempt_no, part in zip(user_ids,targets, timestamps, attempt_nos, parts):
#             lagtime_sum_dict[user_id] = timestamp
#             lagtime_count_dict[user_id] += 1
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            attempt_cum_sum_dict[user_id] += attempt_no
            attempt_cum_count_dict[user_id] += 1
            
            # part
#             k = (user_id,part)
            if k in part_user_sum_dict.keys():
                part_user_sum_dict[k] += answered_correctly
                part_user_count_dict[k] += 1
            else:
                part_user_sum_dict[k] = answered_correctly
                part_user_count_dict[k] = 1
                
#             user_sum_dict2[user_id] += answered_correctly
#             user_count_dict2[user_id] += 1
        del user_ids, targets
            
    test_df["attempt_no"] = test_df[["user_id", "content_id"]].apply(lambda row: get_max_attempt(row["user_id"], row["content_id"]), axis=1)
#     test_df=test_df.merge(questions_df.loc[questions_df.index.isin(test_df['content_id'])],
#                   how='left', on='content_id', right_index=True)
    test_df['join_key'] = test_df['content_type_id'].astype('str') + '_' + test_df['content_id'].astype('str')
    test_df=test_df.merge(content_df.loc[content_df.index.isin(test_df['content_id'])],
                  how='left', on='join_key', right_index=True)
    prior_test_df = test_df.copy() 
    question_len=len(test_df[test_df['content_type_id'] == 0])
    
    
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df.prior_question_had_explanation=test_df.prior_question_had_explanation.astype('int8')
    test_df['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace=True)
    
#     test_df["attempt_no"] = test_df[["user_id", "content_id"]].apply(lambda row: get_max_attempt(row["user_id"], row["content_id"]), axis=1)
    

#     user_lecture_sum = np.zeros(question_len, dtype=np.int16)
#     user_lecture_count = np.zeros(question_len, dtype=np.int16) 
    
#     lagtime_sum = np.zeros(question_len, dtype=np.int16)
#     lagtime_count = np.zeros(question_len, dtype=np.int16)
    
    user_sum = np.zeros(question_len, dtype=np.int16)
    user_count = np.zeros(question_len, dtype=np.int16)
    part_user_sum = np.zeros(question_len, dtype=np.int16)
    part_user_count = np.zeros(question_len, dtype=np.int16)
    part_lecture_user_sum = np.zeros(question_len, dtype=np.int16)
    part_lecture_user_count = np.zeros(question_len, dtype=np.int16)
    attempt_sum = np.zeros(question_len, dtype=np.int16)
    attempt_count = np.zeros(question_len, dtype=np.int16)
#     user_sum2 = np.zeros(question_len, dtype=np.int16)
#     user_count2 = np.zeros(question_len, dtype=np.int16)

#     user_sum_dict_test=defaultdict(int)
#     user_count_dict_test=defaultdict(int)

    task_container_sum = np.zeros(question_len, dtype=np.int32)
    task_container_count = np.zeros(question_len, dtype=np.int32)
    task_container_std = np.zeros(question_len, dtype=np.float16)

    explanation_sum = np.zeros(question_len, dtype=np.int32)
    explanation_count = np.zeros(question_len, dtype=np.int32)
#     delta_prior_question_elapsed_time = np.zeros(question_len, dtype=np.int32)

    attempt_no_count = np.zeros(question_len, dtype=np.int16)
    lagtime = np.zeros(question_len, dtype=np.float32)
    lagtime2 = np.zeros(question_len, dtype=np.float32)
    lagtime3 = np.zeros(question_len, dtype=np.float32)
    lagtime_in_lecture = np.zeros(question_len, dtype=np.float32)
    part_lagtime = np.zeros(question_len, dtype=np.float32)
#     content_lagtime = np.zeros(question_len, dtype=np.float32)
    #lagtime_means = np.zeros(question_len, dtype=np.int32)
    #
    
   
    i=0
    for j, (user_id,prior_question_had_explanation,content_type_id,prior_question_elapsed_time,timestamp, content_id,task_container_id, attempt_no, part) in enumerate(zip(test_df['user_id'].values,test_df['prior_question_had_explanation'].values,test_df['content_type_id'].values,test_df['prior_question_elapsed_time'].values,test_df['timestamp'].values, test_df['content_id'].values, test_df['task_container_id'].values, test_df["attempt_no"].values, test_df["part"].values)):
        
         #
#         user_lecture_sum_dict[user_id] += content_type_id
#         user_lecture_count_dict[user_id] += 1
        if(content_type_id==1):#
            x=1
            max_timestamp_u_dict_c['max_time_stamp'][user_id]=timestamp
            k = (user_id, part)
            if k in part_lecture_user_sum_dict.keys():
                part_lecture_user_sum_dict[k] += 1
                part_lecture_user_count_dict[k] += 1
            else:
                part_lecture_user_sum_dict[k] = 1
                part_lecture_user_count_dict[k] = 1
#             if(len(user_lecture_stats_part[user_lecture_stats_part.user_id==user_id])==0):
#                 user_lecture_stats_part = user_lecture_stats_part.append([{'user_id':user_id}], ignore_index=True)
#                 user_lecture_stats_part.fillna(0, inplace=True)
#                 user_lecture_stats_part.loc[user_lecture_stats_part.user_id==user_id,part_lectures_columns + types_of_lectures_columns]+=lectures_df[lectures_df.lecture_id==content_id][part_lectures_columns + types_of_lectures_columns].values
#             else:
#                 user_lecture_stats_part.loc[user_lecture_stats_part.user_id==user_id,part_lectures_columns + types_of_lectures_columns]+=lectures_df[lectures_df.lecture_id==content_id][part_lectures_columns + types_of_lectures_columns].values
        if(content_type_id==0):#   
#             user_lecture_sum[i] = user_lecture_sum_dict[user_id]
#             user_lecture_count[i] = user_lecture_count_dict[user_id]
#             lagtime_sum[i] = lagtime_sum_dict[user_id]
#             lagtime_count[i] = lagtime_count_dict[user_id]
    
            user_sum[i] = user_sum_dict[user_id]
            user_count[i] = user_count_dict[user_id]
            attempt_sum[i] = attempt_cum_sum_dict[user_id]
            attempt_count[i] = attempt_cum_count_dict[user_id]
#             user_sum2[i] = user_sum_dict2[user_id]
#             user_count2[i] = user_count_dict2[user_id]
    #         content_sum[i] = content_sum_dict[content_id]
    #         content_count[i] = content_count_dict[content_id]
            task_container_sum[i] = task_container_sum_dict[task_container_id]
            task_container_count[i] = task_container_count_dict[task_container_id]
            task_container_std[i]=task_container_std_dict[task_container_id]

            explanation_sum_dict[user_id] += prior_question_had_explanation
            explanation_count_dict[user_id] += 1
            explanation_sum[i] = explanation_sum_dict[user_id]
            explanation_count[i] = explanation_count_dict[user_id]
            
            k = (user_id, part)
            if k in max_timestamp_u_part_dict.keys():
                tmp = max_timestamp_u_part_dict[k]
                part_lagtime[i] = timestamp-tmp
                max_timestamp_u_part_dict[k] = timestamp
            else:
                part_lagtime[i] = -1
                max_timestamp_u_part_dict[k] = timestamp
            part_user_sum[i] = part_user_sum_dict[k]
            part_user_count[i] = part_user_count_dict[k]
            
            if k in part_lecture_user_sum_dict.keys():
                part_lecture_user_count_dict[k] += 1
            else:
                part_lecture_user_count_dict[k] = 1
            part_lecture_user_sum[i] = part_lecture_user_sum_dict[k]
            part_lecture_user_count[i] = part_lecture_user_count_dict[k]

            if user_id in max_timestamp_u_dict['max_time_stamp'].keys():
                lagtime[i]=timestamp-max_timestamp_u_dict['max_time_stamp'][user_id]
                lagtime_in_lecture[i]=timestamp-max_timestamp_u_dict_c['max_time_stamp'][user_id]
                if(max_timestamp_u_dict2['max_time_stamp2'][user_id]==lagtime_mean2):#
                    lagtime2[i]=lagtime_mean2
                    lagtime3[i]=lagtime_mean3
                    #max_timestamp_u_dict3['max_time_stamp3'].update({user_id:lagtime_mean3})
                else:
                    lagtime2[i]=timestamp-max_timestamp_u_dict2['max_time_stamp2'][user_id]
                    if(max_timestamp_u_dict3['max_time_stamp3'][user_id]==lagtime_mean3):
                        lagtime3[i]=lagtime_mean3 #
                    else:
                        lagtime3[i]=timestamp-max_timestamp_u_dict3['max_time_stamp3'][user_id]
                    
                    max_timestamp_u_dict3['max_time_stamp3'][user_id]=max_timestamp_u_dict2['max_time_stamp2'][user_id]
                        
                max_timestamp_u_dict2['max_time_stamp2'][user_id]=max_timestamp_u_dict['max_time_stamp'][user_id]
                max_timestamp_u_dict['max_time_stamp'][user_id]=timestamp
#                 lagtime_means[i]=(lagtime_mean_dict[user_id]+lagtime[i])/2
#                 lagtime_mean_dict[user_id]=lagtime_means[i]
            else:
                lagtime[i]=lagtime_mean
                lagtime_in_lecture[i] = lagtime_in_lecture_mean
                max_timestamp_u_dict['max_time_stamp'].update({user_id:timestamp})
                lagtime2[i]=lagtime_mean2#
                max_timestamp_u_dict2['max_time_stamp2'].update({user_id:lagtime_mean2})
                lagtime3[i]=lagtime_mean3#
                max_timestamp_u_dict3['max_time_stamp3'].update({user_id:lagtime_mean3})
#                 lagtime_mean_dict.update({user_id:lagtime_mean})
#                 lagtime_means[i]=(lagtime_mean_dict[user_id]+lagtime[i])/2
            
#             tmp = max_timestamp_u_content[(max_timestamp_u_content.user_id==user_id) & (max_timestamp_u_content.content_id==content_id)]
#             if len(tmp)==1:
#                 content_lagtime[i] = timestamp - tmp.timestamp.values[0]
#                 max_timestamp_u_content.loc[(max_timestamp_u_content.user_id==user_id) & (max_timestamp_u_content.content_id==content_id), 'timestamp'] = timestamp
#             else:
#                 content_lagtime[i] = -1
#                 max_timestamp_u_content.append({'user_id': user_id, 'content_id': content_id, 'timestamp': timestamp}, ignore_index=True)

            if user_id in user_prior_question_elapsed_time_dict['prior_question_elapsed_time'].keys():            
#                 delta_prior_question_elapsed_time[i]=prior_question_elapsed_time-user_prior_question_elapsed_time_dict['prior_question_elapsed_time'][user_id]
                user_prior_question_elapsed_time_dict['prior_question_elapsed_time'][user_id]=prior_question_elapsed_time
            else:           
#                 delta_prior_question_elapsed_time[i]=delta_prior_question_elapsed_time_mean    
                user_prior_question_elapsed_time_dict['prior_question_elapsed_time'].update({user_id:prior_question_elapsed_time})
            i=i+1 


        
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    
    #HDKIM SAKT
    test_dataset = TestDataset(group, test_df, n_skill, max_seq=MAX_SEQ)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    SAKT_outs = []
    
    for item in test_dataloader:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()

        with torch.no_grad():
            output, att_weight = sakt_model(x, target_id)
 
        output = torch.sigmoid(output)
        output = output[:, -1]
        SAKT_outs.extend(output.view(-1).data.cpu().numpy())
    
    #test_df = test_df[test_df['content_type_id'] == 0]
    #right_index=True
    #test_df = pd.merge(test_df, questions_df, on='content_id', how='left',right_index=True)    
    #test_df = pd.concat([test_df.reset_index(drop=True), questions_df.reindex(test_df['content_id'].values).reset_index(drop=True)], axis=1)
    test_df=test_df.merge(questions_df.loc[questions_df.index.isin(test_df['content_id'])],
                  how='left', on='content_id', right_index=True)
    
    #test_df = pd.merge(test_df, user_lecture_stats_part, on=['user_id'], how="left",right_index=True)
    #test_df = pd.concat([test_df.reset_index(drop=True), user_lecture_stats_part.reindex(test_df['user_id'].values).reset_index(drop=True)], axis=1)
#     test_df=test_df.merge(user_lecture_stats_part.loc[user_lecture_stats_part.index.isin(test_df['user_id'])],
#                   how='left', on='user_id', right_index=True)
 
#     test_df['user_lecture_lv'] = user_lecture_sum / user_lecture_count
#     test_df['user_lecture_sum'] = user_lecture_sum
    
#     test_df['user_interaction_count'] = user_lecture_count
#     test_df['user_interaction_timestamp_mean'] = test_df['timestamp']/user_lecture_count

#     test_df['lagtime_cumsum'] = lagtime_sum
#     test_df['user_interaction_lagtime_mean'] = lagtime_sum / lagtime_count
    
    test_df['user_correctness'] = user_sum / user_count
    test_df['user_uncorrect_count'] =user_count-user_sum
    test_df['user_correct_count'] =user_sum
    test_df['attempt_no_mean'] = attempt_sum / attempt_count
    test_df['user_part_correctness'] = part_user_sum / part_user_count
    test_df['user_part_rate'] = part_user_count / user_count
    test_df['user_part_correctness'] = part_user_sum / part_user_count
    test_df['user_part_rate'] = part_user_count / user_count
    test_df['user_lecture_sum'] = part_lecture_user_sum
    test_df['user_lecture_lv'] = part_lecture_user_sum / part_lecture_user_count
    
    #test_df['user_answer_count'] =user_count
    
#     test_df['user_correctness2'] = user_sum2 / user_count2
#     test_df['user_uncorrect_count2'] =user_count2-user_sum2
#     test_df['user_correct_count2'] =user_sum2
    #test_df['user_answer_count2'] =user_count2
    
    #    
#     test_df['task_container_correctness'] = task_container_sum / task_container_count
#     test_df['task_container_cor_count'] = task_container_sum 
    test_df['task_container_uncor_count'] =task_container_count-task_container_sum 
#     test_df['task_container_std'] = task_container_std 
    #test_df['content_task_mean'] = content_task_mean 
    
#     test_df['explanation_mean'] = explanation_sum / explanation_count
#     test_df['explanation_true_count'] = explanation_sum
    test_df['explanation_false_count'] = explanation_count-explanation_sum 
    
    #
#     test_df['delta_prior_question_elapsed_time'] = delta_prior_question_elapsed_time 
    
  
    # test_df["attempt_no"] = test_df[["user_id", "content_id"]].apply(lambda row: get_max_attempt(row["user_id"], row["content_id"]), axis=1)
    test_df["lagtime"]=lagtime
    test_df["lagtime2"]=lagtime2
    test_df["lagtime3"]=lagtime3
    test_df["lagtime_in_lecture"]=lagtime_in_lecture
    test_df['part_lagtime'] = part_lagtime
    test_df["content_lagtime"] = test_df[["user_id", "content_id", "timestamp"]].apply(lambda row: get_content_lagtime(row["user_id"], row["content_id"], row["timestamp"]), axis=1)
    
    #test_df["lagtime_mean"]=lagtime_means
    test_df['lag_elapsed_time'] = test_df['lagtime'] - test_df['prior_question_elapsed_time']
    test_df['lag_elapsed_time'].fillna(lag_elapsed_time_mean, inplace=True)

    

    test_df['timestamp']=test_df['timestamp']/(1000*3600)
    test_df.timestamp=test_df.timestamp.astype('float16')
    test_df['lagtime']=test_df['lagtime']/(1000*3600)
    test_df.lagtime=test_df.lagtime.astype('float32')
    test_df['lagtime2']=test_df['lagtime2']/(1000*3600)
    test_df.lagtime2=test_df.lagtime2.astype('float32')
    test_df['lagtime3']=test_df['lagtime3']/(1000*3600)
    test_df.lagtime3=test_df.lagtime3.astype('float32')
    test_df['lagtime_in_lecture']=test_df['lagtime_in_lecture']/(1000*3600)
    test_df.lagtime_in_lecture=test_df.lagtime_in_lecture.astype('float32')
    test_df['content_lagtime']=test_df['content_lagtime']/(1000*3600)
    test_df.content_lagtime=test_df.content_lagtime.astype('float32')
    test_df['part_lagtime']=test_df['part_lagtime']/(1000*3600)
    test_df.part_lagtime=test_df.part_lagtime.astype('float32')
    
    
    test_df['lagtime1_2'] = test_df['lagtime'] - test_df['lagtime2']
    test_df['lagtime2_3'] = test_df['lagtime2'] - test_df['lagtime3']
    
    
#     test_df['user_interaction_timestamp_mean']=test_df['user_interaction_timestamp_mean']/(1000*3600)
#     test_df.user_interaction_timestamp_mean=test_df.user_interaction_timestamp_mean.astype('float32')
    
#     test_df['user_interaction_lagtime_mean'] = test_df["lagtime"]/user_count
#     test_df['user_interaction_lagtime_mean'] = test_df['user_interaction_lagtime_mean']/(1000*3600)
#     test_df.user_interaction_lagtime_mean = test_df.user_interaction_lagtime_mean.astype('float32')
    
    test_df['user_correctness'].fillna(0.67, inplace=True)
    #test_df['user_correctness2'].fillna(0.67, inplace=True)
    #
    #test_df = test_df.astype(features_dict)

    sub_preds = np.zeros(test_df.shape[0])
    for i, model in enumerate(clfs, 1):
        test_preds  = model.predict(test_df[features])
        sub_preds += test_preds
    lgbm_final = sub_preds / len(clfs)
    test_df[target] = np.array(SAKT_outs) * 0.5 + lgbm_final * 0.5
#     test_df[target] = lgbm_final
    
#     if(flag_lgbm):
#         test_df[target] = model.predict(test_df[features])
#     else:
#         test_df[target] = model.predict(test_df[features].values)
    env.predict(test_df[['row_id', target]])

pre:  18
aft:  18
pre:  27
aft:  27
pre:  26
aft:  26
pre:  33
aft:  33
CPU times: user 1.82 s, sys: 728 ms, total: 2.54 s
Wall time: 1.4 s
